Домашнее задание №9 делаем на основе задания №5 (там уже была реализована RAG система с ретривером - hw5.ipynb)

In [4]:
%pip install langfuse
%pip install langchain langgraph langchain_openai langchain_community
%pip install --upgrade --quiet  yandexcloud yandex-cloud-ml-sdk
%pip install ragas==0.3.4


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Создаем словарь документов, обогащаем полным текстом из md файлов, переводим в датафрейм и разбиваем на части.

In [5]:
import os
import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS



documents = [
    {
        "id": 166213,
        "title": "Изменить формат общего шаблона excel при скачивании файлов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/"
    },

    {
        "id": 151543,
        "title": "Реализовать страницу со списком акселераторов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/"
    },

    {
        "id": 196677,
        "title": "[Bug][Major] Доработать статусы заявок в Акселераторах",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/196677/"
    },

    {
        "id": 181744,
        "title": "Реализация группы прав \"Менеджер Акселератора партнер\"",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/181744/"
    },

    {
        "id": 190897,
        "title": "Доработка кабинета Акселератора, добавление внешних модераторов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/190897/"
    },

    {
        "id": 184464,
        "title": "Доработка ролевой модели в Акселерации (Демо)",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/184464/"
    },

    {
        "id": 187834,
        "title": "Добавление рассылки видеозаписи прошедшего мероприятия",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/"
    },
]


# Добавляем поле context с текстом из .md файлов
for doc in documents:
    file_name = f"{doc['id']}.md"
    file_path = os.path.join("tasks", file_name)

    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            doc["context"] = f.read()
    else:
        doc["context"] = ""

    # Создаем поле search как объединение title и context
    doc["search"] = f"{doc['title']} {doc['context']}"


# создаем из наших документов датафрейм
df = pd.DataFrame(documents)

# определяем колонку для поиска
loader = DataFrameLoader(df, page_content_column='search')
documents = loader.load()


# создаем сплиттер документов, чтобы уложиться в лимит по токенам
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(texts)

[Document(metadata={'id': 166213, 'title': 'Изменить формат общего шаблона excel при скачивании файлов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/', 'context': 'Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'}, page_content='Изменить формат общего шаблона excel при скачивании файлов Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'), Document(metadata={'id': 151543, 'title': 'Реализовать страницу со списком акселераторов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/', 'context': '1. Необходимо сделать активной кнопку "Акселерация" в разделе Бизнес-комп

In [6]:
import os

YC_FOLDER_ID = os.getenv("folder_id")
YC_API_KEY = os.getenv("ya_token")


Создаем индекс FAISS на основе YandexGPTEmbeddings

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Создаем модель для генерации эмбеддингов
yandex_embeddings = YandexGPTEmbeddings(model="doc", folder_id=YC_FOLDER_ID, iam_token=YC_API_KEY)

# Проверяем наличие локального индекса
index_path = "faiss_index"
if os.path.exists(index_path):
    print(f"Загрузка существующего индекса из {index_path}")
    faiss_index = FAISS.load_local(index_path, yandex_embeddings, allow_dangerous_deserialization=True)
else:
    print("Создание нового индекса")
    # Создание векторного хранилища FAISS
    faiss_index = FAISS.from_texts(
        texts=[doc.page_content for doc in texts],  # Тексты для индексирования
        embedding=yandex_embeddings,  # Экземпляр Embeddings
        metadatas=[{"id": doc.metadata.get("id", ""), "url": doc.metadata.get("url", ""),
                    "title": doc.metadata.get("title", "")} for doc in texts]  # Метаданные
    )
    # Сохранение индекса локально
    faiss_index.save_local(index_path)


Загрузка существующего индекса из faiss_index


Создаем основной словарь с вопросами для тестирования llm

In [8]:
questions = {
    "q1": {
        "text": "На какую страницу попадает пользователь при переходе из раздела «Акселерация»?",
        "category": "navigation",
        "difficulty": "easy"
    },
    "q2": {
        "text": "Какие возможности доступны пользователю с группой прав «Менеджер Акселератора партнёр» во вкладке «Кабинет Акселератора»?",
        "category": "permissions",
        "difficulty": "medium"
    },
    "q3": {
        "text": "Каким образом будут отправляться уведомления о записи прошедшего мероприятия?",
        "category": "notifications",
        "difficulty": "easy"
    },
    "q4": {
        "text": "Какие поля доступны в карточке мероприятия?",
        "category": "data_structure",
        "difficulty": "medium"
    },
    "q5": {
        "text": "Какое ограничение существует для поля «Модератор мероприятий внешний»?",
        "category": "restrictions",
        "difficulty": "hard"
    },
    "q6": {
        "text": "Пользователям из какой группы доступно добавление в качестве внешних модераторов?",
        "category": "permissions",
        "difficulty": "medium"
    },
    "q7": {
        "text": "Какие могут быть статусы заявок в уведомлениях в «колокольчике»?",
        "category": "status",
        "difficulty": "easy"
    }
}

Создаем функцию получения ответа от llm на основе запроса и faiss индекса

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def answer_question(llm, query):
    """
    Отвечает на вопрос с использованием RAG подхода

    Args:
        llm: Модель LLM (например YandexGPT)
        query: Входной вопрос
        faiss_index: Векторный индекс FAISS

    Returns:
        Текст ответа модели
    """
    # Шаблон промпта
    prompt_template = PromptTemplate(
        input_variables=["question", "context"],
        template="""Вы — фактологичный помощник. Отвечайте кратко и точно.
Используйте следующие документы:

Контекст:
{context}

Вопрос:
{question}

Ответ:"""
    )

    # Создаем цепочку
    llm_chain = LLMChain(llm=llm, prompt=prompt_template)

    # Получаем релевантные документы
    relevants = faiss_index.similarity_search(query, k=2)

    # Формируем контекст
    context = "\n\n".join([doc.page_content for doc in relevants])

    # Выполняем запрос
    result = llm_chain.invoke({
        "question": query,
        "context": context
    })

    return result["text"]

Добавляем в тестовые вопросы правильные ответы на основе ответов старшей модели YandexGpt

Результат сохраняем в файл

Пишем служебную функцию для обновления полей словаря questions и сохранения промежуточных данных в файл

In [10]:
import os
import json

def save_questions_to_file(
    filename: str,
    field_name: str,
    llm,
    questions: dict
) -> dict:
    """
    Сохраняет/обновляет вопросы в файл с генерацией указанного поля

    Args:
        filename: Имя файла для сохранения
        field_name: Имя поля для генерации (например, "ground_truth")
        llm: Модель LLM для генерации значений
        questions: Исходный словарь вопросов

    Returns:
        Обновленный словарь вопросов
    """

    # Проверяем существование файла
    if os.path.exists(filename):
        print(f"Загрузка вопросов из {filename}")
        with open(filename, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Проверяем наличие поля в существующих данных
        has_field = any(field_name in q for q in data.values())

        if not has_field:
            print(f"Генерация {field_name} для вопросов...")
            for key, q in data.items():
                q[field_name] = answer_question(llm, q["text"])

            # Сохраняем обновленные данные
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

        return data

    else:
        print(f"Сохранение новых вопросов в {filename}")
        # Генерируем новые значения
        for key, q in questions.items():
            q[field_name] = answer_question(llm, q["text"])

        # Сохраняем в файл
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(questions, f, ensure_ascii=False, indent=4)

        return questions

Создаем модели и заполняем словарь questions

In [11]:
from langchain_community.llms import YandexGPT
import os
import json

# Создаем старшую модель для правильных ответов
big_llm = YandexGPT(
    folder_id=YC_FOLDER_ID,
    iam_token=YC_API_KEY,
    model="yandexgpt",
    temperature=0.3,
    max_tokens=500
)

# Создаем младшую модель для реальных ответов
small_llm = YandexGPT(
    folder_id=YC_FOLDER_ID,
    iam_token=YC_API_KEY,
    model="yandexgpt-lite",
    temperature=0.3,
    max_tokens=500
)

# Путь к файлу c эталонными ответами старшей модели
QUESTIONS_FILE_WITH_GROUND = "questions_ground_truth.json"

# Добавляем в question поле с эталонными ответами
questions = save_questions_to_file(
    filename=QUESTIONS_FILE_WITH_GROUND,
    field_name="ground_truth",
    llm=big_llm,
    questions=questions
)

# Путь к файлу c ответами младшей модели
QUESTIONS_FILE_WITH_SMALL_RESPONSE = "questions_small_response.json"

# Добавляем в question поле с ответами младшей модели
questions = save_questions_to_file(
    filename=QUESTIONS_FILE_WITH_SMALL_RESPONSE,
    field_name="response",
    llm=small_llm,
    questions=questions
)



Загрузка вопросов из questions_ground_truth.json
Загрузка вопросов из questions_small_response.json


Расчитываем метрики RAGAS для оценки результатов младшей модели

In [31]:
results = {}

for key, q in questions.items():
    results[key] = {
        "question": q["text"],
        "response": q["response"],
        "ground_truth": q.get("ground_truth", "")
    }

# Формируем датасет
df_eval = pd.DataFrame([
    {
        "question": r["question"],
        "retrieved_contexts": [doc.page_content for doc in faiss_index.similarity_search(r["question"], k=2)],  # список релевантных документов
        "answer": r["response"],
        "ground_truth": r["ground_truth"]
    }
    for r in results.values()
])

from ragas import evaluate
from ragas.metrics import Faithfulness, ContextPrecision, ContextRecall
from datasets import Dataset

evaluator_embeddings = yandex_embeddings
hf_ds = Dataset.from_pandas(df_eval)

# Проверка структуры данных
print("Пример структуры данных:")
print(hf_ds[0])

# Вычисление метрик
metrics = [Faithfulness(), ContextPrecision(), ContextRecall()]

result = evaluate(
    dataset=hf_ds,
    metrics=metrics,
    llm=small_llm,  # Используем младшую модель для оценки
    embeddings=evaluator_embeddings,
    show_progress=True,
    raise_exceptions=True,
    batch_size=4,
)

print("\nРезультаты оценки RAGAS:")
print(result)

Структура датасета:
{'question': 'На какую страницу попадает пользователь при переходе из раздела «Акселерация»?', 'retrieved_contexts': ['Реализовать страницу со списком акселераторов 1. Необходимо сделать активной кнопку "Акселерация" в разделе Бизнес-компетенций\n2. При переходе из раздела Акселерация -пользователь попадает на страницу списка акселераторов\nСсылка на страницу со списком: https://www.figma.com/file/d1UJj9qDws3...D0%B0%D...\n\n3. Кнопка подачи заявки на акселератор с текстовой подсказкой (Отображение кнопок описано в задаче: https://inside.notamedia.ru/ex/workgro...ew/160210/)\n\nПоля, влияющие на отображение превью карточки акселератора (Поля описаны в задаче в блоке "Значения свойств"\n]https://inside.notamedia.ru/ex/workgroups/group/3974/tasks/task/view/159946/?EVENT_TYPE=UPDATE&a...= ):\n\nПоле админ.панели\n\t\n\nВлияние\n\nАктивность\n\t\n\nОтображение карточки в списке   акселераторов\n\nДата начала активности\n\t\n\nНачало отображения карточки акселератора   в

Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

Batch 1/6:   0%|          | 0/4 [00:00<?, ?it/s]


Результаты оценки RAGAS:
{'faithfulness': 0.8750, 'context_precision': 0.9286, 'context_recall': 1.0000}
